## Import Libraries and Setup

In [1]:
# Standard library imports
import warnings

# Third-party imports
import numpy as np
import pandas as pd

# Scikit-learn imports
from sklearn.metrics import (
    auc,
    precision_recall_curve,
    roc_curve,
)

# CatBoost imports
from catboost import (
    CatBoostClassifier,
    Pool,
)

# Optuna imports
import optuna.visualization as vis
from optuna import Trial, create_study
from optuna.exceptions import ExperimentalWarning

from optuna.integration import CatBoostPruningCallback
from optuna.pruners import MedianPruner
from optuna.storages import JournalStorage, JournalFileStorage

# Local/project imports
from test_case_fairmoney.utils.constants import (
    PATH_DATA_COMPRESSED,
    PATH_DATA_COMPRESSED,
)
from test_case_fairmoney.utils.tunning import run_optimization

# Configuration
pd.set_option("display.max_rows", 300)
pd.set_option("display.max_columns", 800)
warnings.filterwarnings("ignore")

## Load Data for Hyperparameter Tuning

Load datasets with a subset of training data for efficient hyperparameter search. Using 5M samples provides a good balance between training speed and model performance estimation.

In [17]:
# Load preprocessed datasets
print("Loading preprocessed datasets...")
df_train = pd.read_parquet(PATH_DATA_COMPRESSED / "df_train.parquet")
df_val = pd.read_parquet(PATH_DATA_COMPRESSED / "df_val.parquet")
df_test = pd.read_parquet(PATH_DATA_COMPRESSED / "df_test.parquet")
df_train["is_checking_negative"] = (df_train["checking_balance"] < 0).astype(int)
df_val["is_checking_negative"] = (df_val["checking_balance"] < 0).astype(int)
df_test["is_checking_negative"] = (df_test["checking_balance"] < 0).astype(int)
print(f"✅ Datasets loaded successfully!")
print(f"   Train set: {df_train.shape}")
print(f"   Validation set: {df_val.shape}")
print(f"   Test set: {df_test.shape}")

Loading preprocessed datasets...
✅ Datasets loaded successfully!
   Train set: (600, 27)
   Validation set: (200, 27)
   Test set: (200, 27)


In [18]:
feat_num = ["checking_balance", "amount", "savings_balance", "employment_months"]
feat_cat = [
    "credit_history",
    "personal_status",
    "property",
    "installment_plan",
    "is_checking_negative",
]
target = "default"

In [19]:
for feature in feat_cat:
    df_train[feature] = df_train[feature].fillna("Missing")
    df_val[feature] = df_val[feature].fillna("Missing")
    df_test[feature] = df_test[feature].fillna("Missing")

## Feature Selection for Optimization

Define the feature set for CatBoost hyperparameter tuning. We focus on the most impactful categorical features identified in previous analysis.

## Prepare CatBoost Data Pools

Create CatBoost Pool objects for efficient data handling during hyperparameter optimization.

In [35]:
# Create initial data pools
train_pool_finetune = Pool(
    data=df_train[feat_num + feat_cat],
    label=df_train[target],
    cat_features=feat_cat,
)
test_pool_finetune = Pool(
    data=df_test[feat_num + feat_cat],
    label=df_test[target],
    cat_features=feat_cat,
)
val_pool_finetune = Pool(
    data=df_val[feat_num + feat_cat],
    label=df_val[target],
    cat_features=feat_cat,
)

## Hyperparameter Optimization Setup

### Define Base Parameters and Search Space

Set up the hyperparameter optimization framework with Optuna, including base parameters, search space definition, and evaluation metrics.

In [ ]:
results = run_optimization(
    train_pool=train_pool_finetune,
    val_pool=val_pool_finetune,
    target_column=target,
    study_name="catboost_auc_optimization_v2",
    storage_file="optuna-journal.log",
    n_trials=500,
)

[I 2025-11-01 16:28:09,705] A new study created in Journal with name: catboost_auc_optimization_v2


  0%|          | 0/500 [00:00<?, ?it/s]

[I 2025-11-01 16:28:09,934] Trial 0 finished with value: 0.7437499999999999 and parameters: {'learning_rate': 0.07024939435329727, 'depth': 7, 'l2_leaf_reg': 18.33796969921668, 'min_data_in_leaf': 15, 'random_strength': 7.991762494366979, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 0.2625811925436583}. Best is trial 0 with value: 0.7437499999999999.
[I 2025-11-01 16:28:10,375] Trial 1 finished with value: 0.7676190476190476 and parameters: {'learning_rate': 0.07127419916775114, 'depth': 6, 'l2_leaf_reg': 1.373385677782168, 'min_data_in_leaf': 40, 'random_strength': 2.2187575633048473, 'boosting_type': 'Ordered', 'bootstrap_type': 'Bernoulli', 'subsample': 0.6751651844700399}. Best is trial 1 with value: 0.7676190476190476.
[I 2025-11-01 16:28:10,505] Trial 2 finished with value: 0.7267857142857143 and parameters: {'learning_rate': 0.03521335573804211, 'depth': 6, 'l2_leaf_reg': 6.032869666578389, 'min_data_in_leaf': 7, 'random_strength': 7.267037844

In [46]:
results["best_params"]

{'learning_rate': 0.02814097068151777,
 'depth': 6,
 'l2_leaf_reg': 27.45959338993318,
 'min_data_in_leaf': 5,
 'random_strength': 4.360675824806039,
 'boosting_type': 'Ordered',
 'bootstrap_type': 'Bayesian',
 'bagging_temperature': 6.457046635415198}

## Optimization Analysis and Visualization

### Optimization History

Visualize how the optimization progressed over trials to understand convergence patterns.

In [38]:
# Alternative 1: Optimization history (works with fewer trials)
fig = vis.plot_optimization_history(results["study"])
fig.show()

# Alternative 2: Parameter importance
fig = vis.plot_param_importances(results["study"])
fig.show()

# Alternative 3: Slice plot
fig = vis.plot_slice(results["study"])
fig.show()

# Alternative 4: EDF plot
fig = vis.plot_edf(results["study"])
fig.show()

In [39]:
# Get parameter names from completed trials
completed_trials = [t for t in results["study"].trials if t.state.name == "COMPLETE"]
if completed_trials:
    param_names = list(completed_trials[0].params.keys())
    fig = vis.plot_parallel_coordinate(results["study"], params=param_names)
    fig.show()